# CDF Response Time Analysis - Closed Network

This example demonstrates cumulative distribution function (CDF) analysis for response times in a closed queueing network.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import jline.solvers.fluid.*
import jline.util.matrix.*
import kotlin.math.*

In [ ]:
// Create closed queueing network with Delay and Queue nodes
val model = Network("model")

val delay = Delay(model, "Delay")
val queue = Queue(model, "Queue2", SchedStrategy.PS)

// Single closed class with 1 job starting at Delay node
val jobclass = ClosedClass(model, "Class1", 1, delay, 0)

In [ ]:
// Service processes - match theoretical values
val servProc1 = Exp(1.0 / 0.1)  // rate = 1/0.1 = 10, mean = 0.1
delay.setService(jobclass, servProc1)

val servProc2 = Erlang.fitMeanAndSCV(1.0, 1.0/3.0)  // mean=1, SCV=1/3
queue.setService(jobclass, servProc2)

// Routing matrix - circular routing between the two nodes
val P = model.initRoutingMatrix()
P.set(jobclass, jobclass, delay, queue, 1.0)
P.set(jobclass, jobclass, queue, delay, 1.0)
model.link(P)

In [ ]:
// Solve with JMT (simulation) solver first
val jmtSolver = JMT(model, "seed", 23000)
val jmtCDF = jmtSolver.cdfRespT

println("JMT Solver CDF Analysis:")
println("CDF structure: ${jmtCDF.size} stations, ${jmtCDF[0].size} classes")

for (i in jmtCDF.indices) {
    for (c in jmtCDF[i].indices) {
        if (jmtCDF[i][c] != null) {
            println("  Station $i, Class $c: ${jmtCDF[i][c].size(0)} CDF points")
        } else {
            println("  Station $i, Class $c: No CDF data")
        }
    }
}

In [ ]:
// Calculate mean response time from CDF (JMT)
val numStations = model.numberOfStations
val numClasses = model.numberOfClasses
val avgRespTfromCDFSim = Matrix(numStations, numClasses)
val sqCoeffOfVariationRespTfromCDFSim = Matrix(numStations, numClasses)

for (i in 0 until numStations) {
    for (c in 0 until numClasses) {
        if (jmtCDF[i][c] != null && jmtCDF[i][c].size(0) > 1) {
            val cdfData = jmtCDF[i][c]
            
            // Calculate mean from CDF using numerical integration
            var meanRT = 0.0
            var secondMoment = 0.0
            
            for (k in 1 until cdfData.size(0)) {
                val dt = cdfData.get(k, 0) - cdfData.get(k-1, 0)  // time difference
                val rt = cdfData.get(k, 1)  // response time value
                meanRT += dt * rt
                secondMoment += dt * rt * rt
            }
            
            val variance = secondMoment - meanRT * meanRT
            val scv = if (meanRT > 0) variance / (meanRT * meanRT) else 0.0
            
            avgRespTfromCDFSim.set(i, c, meanRT)
            sqCoeffOfVariationRespTfromCDFSim.set(i, c, scv)
        }
    }
}

In [ ]:
// Solve with Fluid solver
val fluidSolver = FLD(model)
val fluidCDF = fluidSolver.cdfRespT

println("\nFluid Solver CDF Analysis:")
println("CDF structure: ${fluidCDF.size} stations, ${fluidCDF[0].size} classes")

for (i in fluidCDF.indices) {
    for (c in fluidCDF[i].indices) {
        if (fluidCDF[i][c] != null) {
            println("  Station $i, Class $c: ${fluidCDF[i][c].size(0)} CDF points")
        } else {
            println("  Station $i, Class $c: No CDF data")
        }
    }
}

In [ ]:
// Calculate theoretical values from service processes
val avgRespTfromTheory = Matrix.of(doubleArrayOf(servProc1.mean, servProc2.mean))
val sqCoeffOfVariationRespTfromTheory = Matrix.of(doubleArrayOf(servProc1.sCV, servProc2.sCV))

println("\nComparison of Response Time Statistics:")
println("======================================")
println("Since there is a single job, mean and squared coefficient of variation")
println("of response times should be close to those of the service time distribution.\n")

println("Theoretical Mean Response Times:")
avgRespTfromTheory.print()

println("\nMean Response Times from JMT CDF:")
avgRespTfromCDFSim.print()

println("\nTheoretical Squared Coefficient of Variation:")
sqCoeffOfVariationRespTfromTheory.print()

println("\nSquared Coefficient of Variation from JMT CDF:")
sqCoeffOfVariationRespTfromCDFSim.print()

This example demonstrates:

1. **CDF Analysis**: Computing cumulative distribution functions for response times
2. **Single Job System**: With one job, response time statistics equal service time statistics
3. **Multiple Solvers**: Comparison between JMT (simulation) and Fluid (analytical approximation)
4. **Statistical Moments**: Calculation of mean and coefficient of variation from CDFs

**Key Concepts**:
- **Service Processes**: 
  - Delay: Exponential with mean 0.1 (rate 10)
  - Queue: Erlang with mean 1.0 and SCV 1/3
- **Single Job**: Response time = Service time (no queueing delays)
- **CDF Integration**: Numerical integration to compute moments from distribution

**Expected Results**:
- Mean response times should match service time means
- Coefficients of variation should match service time SCVs
- JMT provides simulation-based estimates
- Fluid provides analytical approximations

This analysis is useful for understanding the full distribution of response times, not just the mean values provided by standard performance metrics.